In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
sample = pd.read_csv("sample_submission.csv")
sample_df = pd.DataFrame(sample)
sample_df.head()

,row_id,document,token,label
0,0,7,9,B-NAME_STUDENT
1,1,7,10,I-NAME_STUDENT
2,2,7,482,B-NAME_STUDENT
3,3,7,483,I-NAME_STUDENT
4,4,7,741,B-NAME_STUDENT


In [3]:
train = pd.read_json("train.json")
train_df = pd.DataFrame(train)
train_df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [4]:
test = pd.read_json("test.json")
test_df = pd.DataFrame(test)
test_df.head()

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."


In [5]:
class MostFrequentClass:
    
    def __init__(self, data: pd.DataFrame, target: str):
        self.target = target
        self.data = data
        self.pred_col1 = []
        self.pred_col2 = []
        self.values_dict = {}
        
    def predict(self):
        col = self.data[self.target]
        
        for row_label_list in [x for x in col]:
            for value in row_label_list:
                self.values_dict[value] = self.values_dict.get(value, 0) + 1
        most_frequent = max(self.values_dict, key=lambda k: self.values_dict[k])
        
        self.data['most_frequent_class'] = pd.Series([most_frequent]*len(row_label_list) for row_label_list in [x for x in col])
            
        values_dict = self.values_dict
        if len(list(values_dict.keys())) > 1:
            values_dict.pop(most_frequent)
        second_frequent = max(values_dict, key=lambda k: values_dict[k])
        
        self.data['next_most_frequent_class'] = pd.Series([second_frequent]*len(row_label_list) for row_label_list in [x for x in col])
        
    def test_predict(self, test: pd.DataFrame):
        values_dict = {}
        col = test[self.target]
        for row_label_list in [x for x in col]:
            for value in row_label_list:
                values_dict[value] = values_dict.get(value, 0) + 1
            most_frequent = max(values_dict, key=lambda k: values_dict[k])
            self.pred_col1.append(most_frequent)
            if len(list(values_dict.keys())) > 1:
                values_dict.pop(most_frequent)
            second_frequent = max(values_dict, key=lambda k: values_dict[k])
            self.pred_col2.append(second_frequent)
            values_dict = {}
        for row_label_list in [x for x in col]:
            test['most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col1)
            test['next_most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col2)
        return test
        
            
        

In [6]:
#split
def train_test_split(data: pd.DataFrame, train_percent: float):
    if train_percent < 0 or train_percent > 1:
        raise ValueError("train_percent must be between 0 and 1")

    test_percent = 1 - train_percent
    test_set = data[:int(len(data) * test_percent)]
    train_set = data[int(len(data) * test_percent):]
    return train_set, test_set

In [7]:
train_split, test_split = train_test_split(train_df, 0.9)

In [8]:
test_split

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
675,8980,LEARNING LAUNCH\n\nCHALLENGE\n\nIt’s the begin...,"[LEARNING, LAUNCH, \n\n, CHALLENGE, \n\n, It, ...","[True, False, False, False, False, False, True...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
676,8982,Visualization tool – Sanne Aarts Challenge ...,"[Visualization, tool, –, Sanne, Aarts, , Ch...","[True, True, True, True, True, False, True, Tr...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
677,8984,1. Challenge : How do we make our customers ad...,"[1, ., Challenge, :, How, do, we, make, our, c...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
678,8986,MIIND MAPPING\n\nChallenge: As a student I ha...,"[MIIND, MAPPING, \n\n, Challenge, :, , As, a,...","[True, False, False, False, True, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [9]:
MFC = MostFrequentClass(train_split, 'labels')
MFC.predict()
MFC.data

C:\Users\HP\AppData\Local\Temp\ipykernel_21224\1092949815.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['most_frequent_class'] = pd.Series([most_frequent]*len(row_label_list) for row_label_list in [x for x in col])
C:\Users\HP\AppData\Local\Temp\ipykernel_21224\1092949815.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['next_most_frequent_class'] = pd.Series([second_frequent]*len(row_label_list) for row_label_list in [x for x in col])


,document,full_text,tokens,trailing_whitespace,labels,most_frequent_class,next_most_frequent_class
680,8988,We believe that all kids are naturally creativ...,"[We, believe, that, all, kids, are, naturally,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
681,8989,Reflection - Mind Mapping\n\nChallenge\n\nI am...,"[Reflection, -, Mind, Mapping, \n\n, Challenge...","[True, True, True, False, False, False, False,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
682,8993,Visualization\n\nChallenge\n\nSince I am an ev...,"[Visualization, \n\n, Challenge, \n\n, Since, ...","[False, False, False, False, True, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
683,8995,Visualization\n\nChallenge\n\nWhen I worked as...,"[Visualization, \n\n, Challenge, \n\n, When, I...","[False, False, False, False, True, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
684,8996,Visualizations are one of the important parts ...,"[Visualizations, are, one, of, the, important,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
...,...,...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN,NaN


In [10]:
pred = MFC.test_predict(test_split)
pred

C:\Users\HP\AppData\Local\Temp\ipykernel_21224\1092949815.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col1)
C:\Users\HP\AppData\Local\Temp\ipykernel_21224\1092949815.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['next_most_frequent_class'] = pd.Series([score]*len(row_label_list) for score in self.pred_col2)
C:\Users\HP\AppData\Local\Temp\ipykernel_21224\1092949815.py:41: SettingWithCopyWarning: 
A

,document,full_text,tokens,trailing_whitespace,labels,most_frequent_class,next_most_frequent_class
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
...,...,...,...,...,...,...,...
675,8980,LEARNING LAUNCH\n\nCHALLENGE\n\nIt’s the begin...,"[LEARNING, LAUNCH, \n\n, CHALLENGE, \n\n, It, ...","[True, False, False, False, False, False, True...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
676,8982,Visualization tool – Sanne Aarts Challenge ...,"[Visualization, tool, –, Sanne, Aarts, , Ch...","[True, True, True, True, True, False, True, Tr...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-NAME_STUDENT, B-NAME_STUDENT, B-NAME_STUDEN..."
677,8984,1. Challenge : How do we make our customers ad...,"[1, ., Challenge, :, How, do, we, make, our, c...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
678,8986,MIIND MAPPING\n\nChallenge: As a student I ha...,"[MIIND, MAPPING, \n\n, Challenge, :, , As, a,...","[True, False, False, False, True, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
